In [2]:
import pandas as pd
import math
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ativos
tickers = ['ITUB4' , 'BBDC4' ,
           'BBAS3' , 'ABCB4' ,
           'BPAC11' , 'SANB11' ,
           'BRSR6' , 'BAZA3',
           'INBR32' , 'ROXO34',
           'XPBR31' , 'BPAN4',
           'BBSE3' , 'PSSA3' , 
           'CXSE3' , 'IRBR3'
            ]
period = "10y"
interval = "1wk"

In [3]:
data_dict = {}  

for ticker in tickers:
    df = yf.download(ticker+".SA", period=period, interval=interval, auto_adjust=True,progress=False)[['Close']]
    data_dict[ticker] = df  

In [4]:
# grafico
height=1500
width=800

mma = 15
fig = make_subplots(
    rows=math.ceil(len(tickers)/2), cols=2, shared_xaxes=False, 
    subplot_titles=tickers,  
    vertical_spacing=0.1
)

for i, ticker in enumerate(tickers):
    df = data_dict[ticker][('Close', ticker+".SA")]  
    avg = df.rolling(window=mma, min_periods=1).mean()
    
    row = (i // 2) + 1
    col = (i % 2) + 1

    fig.add_trace(go.Scatter(
        x=df.index, y=avg, mode='lines',
        line=dict(color='blue', width=1),
        name="Média", showlegend=False), row=row, col=col)

fig.update_layout(title_text=f"<b>Bancos e Seguradoras</b><br><sub>mm{mma} do fechamento semanal</sub>",
                  height= height, width=width)
fig.update_xaxes(rangeslider_visible=False,
    tickformat="%Y",  # Mostrar apenas o ano
    dtick="M36",  # Intervalo de meses entre os ticks
    ticklabelmode="period"  # Coloca o rótulo no início do período)
)
fig.show()


In [10]:
import requests
from io import StringIO

#pra fingir que é um browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funds = pd.read_html(StringIO(r.text), decimal=',', thousands='.')[0]
dfunds = pd.DataFrame(funds)

media_pl = dfunds[dfunds['Papel'].isin(tickers)]['P/L'].mean()
print(f'\nO P/L médio dessas empresas é {media_pl:.2f}')

print(dfunds[dfunds['Papel'].isin(tickers)]['P/L'].describe())

ibra = pd.read_csv('data/Cart_IBrA.csv', 
                 sep=';',
                 encoding='latin1',
                 engine='python',
                 skiprows=2,    
                 skipfooter=2,
                 usecols=[0],header=None)[0].to_list()
intersecao = sorted(set(tickers) & set(ibra))

media_pl_indice = dfunds[dfunds['Papel'].isin(intersecao)]['P/L'].mean()
print(f'\nO P/L médio dessas empresas que estão no índice amplo ({intersecao}) é {media_pl_indice:.2f}')



O P/L médio dessas empresas é 10.10
count    13.000000
mean     10.103846
std       4.482216
min       4.410000
25%       6.710000
50%       9.660000
75%      11.100000
max      21.470000
Name: P/L, dtype: float64

O P/L médio dessas empresas que estão no índice amplo (['ABCB4', 'BBAS3', 'BBDC4', 'BBSE3', 'BPAC11', 'BRSR6', 'CXSE3', 'IRBR3', 'ITUB4', 'PSSA3', 'SANB11']) é 9.59


In [19]:
def limpar_numero(valor):
    """
    Converte número/porcentagem em qualquer formato para float.
    
    Exemplos:
        "12,34%" → 12.34
        "-2.08%" → -2.08
        "1.234,56" → 1234.56
        "-45.589,10" → -45589.10
    """
    if pd.isna(valor):
        return None
    
    try:
        # Converter para string e remover espaços e %
        num = str(valor).strip().replace('%', '')
        
        # Detectar formato: se tem vírgula, é formato brasileiro
        if ',' in num:
            # Formato brasileiro: 1.234,56
            num = num.replace('.', '')      # Remove separador de milhar
            num = num.replace(',', '.')     # Vírgula vira ponto decimal
        # Senão, já está no formato correto (1234.56 ou -2.08)
        
        return float(num)
    except (ValueError, AttributeError):
        return None


In [20]:
dfunds['ROE'] = dfunds['ROE'].apply(limpar_numero)


In [23]:
media_roe = dfunds[dfunds['Papel'].isin(tickers)]['ROE'].mean()
print(f'\nO ROE médio dessas empresas é {media_roe:.2f}')

print(dfunds[dfunds['Papel'].isin(tickers)]['ROE'].describe())

media_roe_indice = dfunds[dfunds['Papel'].isin(intersecao)]['ROE'].mean()
print(f'\nO ROE médio dessas empresas que estão no índice amplo ({intersecao}) é {media_roe_indice:.2f}')


O ROE médio dessas empresas é 20.74
count    13.000000
mean     20.742308
std      16.236532
min       7.940000
25%      12.130000
50%      14.740000
75%      22.380000
max      69.930000
Name: ROE, dtype: float64

O ROE médio dessas empresas que estão no índice amplo (['ABCB4', 'BBAS3', 'BBDC4', 'BBSE3', 'BPAC11', 'BRSR6', 'CXSE3', 'IRBR3', 'ITUB4', 'PSSA3', 'SANB11']) é 22.29
